# **Importación de Librerías y Autenticación** 📓

In [1]:
import geemap
import ee

In [ ]:
ee.Authenticate()
ee.Initialize(project = "edu-irnr")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Etiquetado final de los datos del INFyS** 🏷

In [ ]:
import pandas as pd
infys_datos = pd.read_csv("/content/drive/MyDrive/TesisDRB2024/Archivos/INFyS+_ML_2018_2019.csv")
print(infys_datos.head(5))
print("")
print(infys_datos.tail(5))
print("")
print(infys_datos.shape)
print("")
print(infys_datos.info())

   UPMID  Anio_C3        X_C3       Y_C3  \
0  50822     2018 -100.202167  18.643028   
1  50822     2018 -100.201917  18.642472   
2  50822     2018 -100.202639  18.642444   
3  50822     2018 -100.202250  18.642611   
4  54188     2018 -100.058083  18.776500   

                                       DESCRIP_S7_C3  \
0                              BOSQUE DE ENCINO-PINO   
1                              BOSQUE DE ENCINO-PINO   
2                              BOSQUE DE ENCINO-PINO   
3                              BOSQUE DE ENCINO-PINO   
4  VEGETACIÓN SECUNDARIA ARBUSTIVA DE BOSQUE DE E...   

                              USV_N  
0  BOSQUE DE ASOCIACIÓN PINO-ENCINO  
1  BOSQUE DE ASOCIACIÓN PINO-ENCINO  
2  BOSQUE DE ASOCIACIÓN PINO-ENCINO  
3  BOSQUE DE ASOCIACIÓN PINO-ENCINO  
4   VEGETACIÓN SECUNDARIA ARBUSTIVA  

     UPMID  Anio_C3       X_C3       Y_C3 DESCRIP_S7_C3   USV_N
266     41     2019 -98.831517  19.702597        URBANO  URBANO
267     42     2019 -98.835502  19.701693

In [ ]:
# Crear un mapeo de categorías únicas a números
categorias_unicas = infys_datos['USV_N'].unique()
mapeo_categorias = {categoria: i+1 for i, categoria in enumerate(categorias_unicas)}

# Aplicar el mapeo a la columna 'USV_N'
infys_datos['USV_N_Clave'] = infys_datos['USV_N'].map(mapeo_categorias)

# Mostrar el resultado
print(infys_datos.head())

   UPMID  Anio_C3        X_C3       Y_C3  \
0  50822     2018 -100.202167  18.643028   
1  50822     2018 -100.201917  18.642472   
2  50822     2018 -100.202639  18.642444   
3  50822     2018 -100.202250  18.642611   
4  54188     2018 -100.058083  18.776500   

                                       DESCRIP_S7_C3  \
0                              BOSQUE DE ENCINO-PINO   
1                              BOSQUE DE ENCINO-PINO   
2                              BOSQUE DE ENCINO-PINO   
3                              BOSQUE DE ENCINO-PINO   
4  VEGETACIÓN SECUNDARIA ARBUSTIVA DE BOSQUE DE E...   

                              USV_N  USV_N_Clave  
0  BOSQUE DE ASOCIACIÓN PINO-ENCINO            1  
1  BOSQUE DE ASOCIACIÓN PINO-ENCINO            1  
2  BOSQUE DE ASOCIACIÓN PINO-ENCINO            1  
3  BOSQUE DE ASOCIACIÓN PINO-ENCINO            1  
4   VEGETACIÓN SECUNDARIA ARBUSTIVA            2  


In [ ]:
## Guardar infys_datos en drive
infys_datos.to_csv("/content/drive/MyDrive/TesisDRB2024/Archivos/infys_datos.csv", index=False)

In [ ]:
## datos unicos de usc_N_CLave
print(infys_datos['USV_N'].unique())
print(infys_datos['USV_N_Clave'].unique())

['BOSQUE DE ASOCIACIÓN PINO-ENCINO' 'VEGETACIÓN SECUNDARIA ARBUSTIVA'
 'VEGETACIÓN SECUNDARIA ARBÓREA' 'PASTIZAL INDUCIDO'
 'BOSQUE MESÓFILO DE MONTAÑA' 'BOSQUE DE OYAMEL' 'BOSQUE DE ENCINO'
 'SELVA BAJA CADUCIFOLIA' 'BOSQUE DE PINO' 'BOSQUE CULTIVADO'
 'AGRICULTURA' 'AGUA' 'URBANO']
[ 1  2  3  4  5  6  7  8  9 10 11 12 13]


In [ ]:
infys_datos.shape

(271, 7)

In [ ]:
# Definir las categorías de uso del suelo y sus colores
landcover_labels = {
    1: "BOSQUE DE ASOCIACIÓN PINO-ENCINO",
    2: "VEGETACIÓN SECUNDARIA ARBUSTIVA",
    3: "VEGETACIÓN SECUNDARIA ARBÓREA",
    4: "PASTIZAL INDUCIDO",
    5: "BOSQUE MESÓFILO DE MONTAÑA",
    6: "BOSQUE DE OYAMEL",
    7: "BOSQUE DE ENCINO",
    8: "SELVA BAJA CADUCIFOLIA",
    9: "BOSQUE DE PINO",
    10: "BOSQUE CULTIVADO",
    11: "AGRICULTURA",
    12: "AGUA",
    13: "URBANO"
}

# Definir los colores para cada categoría
landcover_colors = [
    '006400',  # BOSQUE DE ASOCIACIÓN PINO-ENCINO
    '8B4513',  # VEGETACIÓN SECUNDARIA ARBUSTIVA
    '228B22',  # VEGETACIÓN SECUNDARIA ARBÓREA
    '7CFC00',  # PASTIZAL INDUCIDO
    '2E8B57',  # BOSQUE MESÓFILO DE MONTAÑA
    '556B2F',  # BOSQUE DE OYAMEL
    '8B4513',  # BOSQUE DE ENCINO
    '00FF00',  # SELVA BAJA CADUCIFOLIA
    '006400',  # BOSQUE DE PINO
    '32CD32',  # BOSQUE CULTIVADO
    'FFFF00',  # AGRICULTURA
    '0000FF',  # AGUA
    'FF0000'   # URBANO
]

# Crear una lista de pares de (valor, color) para visualización
landcover_palette = {
    str(index): color for index, color in zip(landcover_labels.keys(), landcover_colors)
}

# Convertir el diccionario a una lista de valores y colores
palette = list(landcover_palette.values())


# **Carga de Datos a GEE** 🌏

In [ ]:
# Definir la fecha de inicio y fin
start_date = '2024-06-01'
end_date = '2024-07-25'

# Cargar la colección de municipios de México
mexico_municipios = ee.FeatureCollection("projects/ee-diego-epp/assets/mun_mex")
infys_gee = ee.FeatureCollection("projects/edu-irnr/assets/infys_datos_F")
# Filtrar por el estado de México
estado_mexico = mexico_municipios.filter(ee.Filter.eq('NOM_ENT', 'Mexico'))

# Cargar la colección de imágenes Sentinel-2 harmonized
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 25)) \
    .filterBounds(estado_mexico)

# Obtener una imagen compuesta utilizando la mediana
image = sentinel2.median()

# Crear un mapa
Map = geemap.Map()

# Centrar el mapa en la región de interés
Map.centerObject(estado_mexico, 8)

# Añadir la imagen compuesta al mapa
vis_params = {
    'bands': ['B4', 'B3', 'B2'],  # RGB
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
ndvi_vis = {
    'min': 0,
    'max': 1,
    'palette': ['red',"yellow", 'lightgreen', 'green']
}

# Añadir la capa NDVI al mapa



# Asegurarnos que nuestros puntos de entrenamiento estén dentro del Estado de México

infys_gee = infys_gee.filterBounds(estado_mexico)

# Añadir mapa base de Satélite
Map.add_basemap("SATELLITE")
Map.addLayer(image.clip(estado_mexico), vis_params, 'Sentinel-2 Jan 2020 (Estado de México)')
Map.addLayer(ndvi.clip(estado_mexico), ndvi_vis, 'NDVI')
# Añadir el contorno del estado de México al mapa
Map.addLayer(estado_mexico, {}, 'Estado de México')
Map.addLayer(infys_gee)

Map(center=[19.246367725884216, -96.45007000915615], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
num_images = sentinel2.size().getInfo()
print(f'Número de imágenes: {num_images}')


Número de imágenes: 3


In [ ]:
image_dates = sentinel2.aggregate_array('system:time_start').getInfo()

import datetime
image_dates = [datetime.datetime.utcfromtimestamp(date / 1000).strftime('%Y-%m-%d') for date in image_dates]
image_dates


['2024-06-01', '2024-06-06', '2024-07-16']

In [ ]:
# Es importante conocer las características de las bandas y la información que aportan al modelo
bands = ["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B9"]

# Definir el nombre de la columna que lleva la clave numérica del tipo de cobertura de la tierra
label = "USV_N_Clave"

In [ ]:
training = image.select(bands).sampleRegions(
    **{"collection": infys_gee, "properties": [label], "scale": 10}
)
trained = ee.Classifier.smileRandomForest(100).train(training, label, bands)

In [ ]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '000000000000000000b7_0', 'properties': {'B1': 343, 'B2': 699, 'B3': 1036, 'B4': 1270, 'B5': 1440.5, 'B6': 1904, 'B7': 2053.5, 'B8': 2366, 'B8A': 2228.5, 'B9': 2420, 'USV_N_Clave': 11}}


In [ ]:
# Clasificar la imagen compuesta utilizando el modelo entrenado
classified = image.select(bands).classify(trained)
classified = classified.clip(estado_mexico)

# Definir los parámetros de visualización
vis_params = {
    'min': 1,
    'max': 13,
    'palette': palette
}

# Crear un mapa
Mapa = geemap.Map()

# Centrar el mapa en la región de interés
Maa.centerObject(estado_mexico, 8)

# Añadir la capa clasificada al mapa
Mapa.addLayer(classified, vis_params, 'Land Cover Classification')

Map(center=[19.355515456288447, -99.64623780096181], controls=(WidgetControl(options=['position', 'transparent…

# **Despliegue del mapa** 🗺

In [ ]:
# Mostrar el mapa
Mapa